In [5]:
import covalent as ct
import time

In [6]:
@ct.electron
def add(x: int, y: int):
    return x + y

@ct.electron
def multiply(x: int, y: int):
    return x*y

@ct.lattice
def workflow(x: int, y: int):
    r2 = 0
    for i in range(10):
        r1 = add(x, y)
        r2 *= multiply(r1, y)
    return r1 + r2

start = time.time()
result = ct.dispatch_sync(workflow)(2, 3)
end = time.time()

print(f"Elapsed: {end-start} seconds")

Elapsed: 3.6850883960723877 seconds


In [ ]:
import numpy as np

@ct.electron
def add(x: int, y: int):
    return x + y

@ct.electron
def multiply(x: int, y: int):
    return x*y

@ct.lattice
def add_workflow(N: int):
    for i in range(N):
        if i == 0:
            r1 = add(1, 1)
        else:
            r1 = add(r1, 1)
    return r1

# normal run
num_tasks=10
start = time.time()
add_res = add_workflow(num_tasks)
end = time.time()
print(f"add_workflow (without covalent): {end-start} seconds")

start = time.time()
add_dispatch_result = ct.dispatch_sync(add_workflow)(num_tasks)
end = time.time()
print(f"add_workflow covalent execution duration: {end-start} seconds")

# Result can potentially overflow
@ct.lattice
def multiply_workflow(N: int):
    for i in range(N):
        if i == 0:
            r1 = multiply(2, 1)
        else:
            r1 = multiply(r1, 2)
    return r1

# normal run
start = time.time()
multiply_res = multiply_workflow(num_tasks)
end = time.time()
print(f"multipy_workflow (without covalent): {end-start} seconds")

start = time.time()
multiply_dispatch_result = ct.dispatch_sync(multiply_workflow)(num_tasks)
end = time.time()
print(f"multiply_workflow execution duration: {end-start} seconds")

# Assert (these are failing)
#assert multiply_res == multiply_dispatch_result.result
#assert add_res == add_dispatch_result.result

In [ ]:
num_tasks = 100
@ct.lattice
def workflow(N: int):
    r2 = 1
    for i in range(N):
        r1 = add(1, 1)
        r2 = multiply(r1, 2)
    return r1 + r2

res = workflow(num_tasks)
start = time.time()
dispatch_res = ct.dispatch_sync(workflow)(num_tasks)
end = time.time()
print(f"List of electrons workflow: {end-start} seconds")

#assert res == dispatch_res.result



In [8]:
@ct.electron
@ct.lattice
def sublattice(x: int, y: int):
    r1 = add(x, y)
    r2 = multiply(r1, y)
    return r1 + r2

@ct.lattice
def main_workflow(num_tasks: int):
    results = []
    for i in range(num_tasks):
        results.append(sublattice(i, i+2))
    return results

N = 100
res = main_workflow(N)
dispatch_res = ct.dispatch_sync(main_workflow)(N)

### Fan in/Fan out workflow

In [7]:
@ct.electron
def add(x, y):
    return x + y

@ct.electron
def multiply(x, y):
    return x*y

@ct.lattice
def workflow(x, y, N):
    r1 = add(x, y)
    for i in range(N):
        r2 = add(r1, y)

    r3 = multiply(r2, x)

    for j in range(N):
        r4 = multiply(r3, y)

    return r1 + r2 + r3 + r4


result = ct.dispatch_sync(workflow)(2, 3, 100)

In [9]:
import torch

In [11]:
def cost(x):
    return torch.sum(x**2)

@ct.electron
def move_forward(optimizer,point):
    loss = cost(point)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return optimizer,point,loss

@ct.lattice
def optimize(point,iterations):
    optimizer = torch.optim.Adam([point], lr=.1)
    for _ in range(iterations):
        optimizer,point,loss=move_forward(optimizer,point)
    return point
      
p=torch.tensor([1,.2], requires_grad=True)
j=ct.dispatch(optimize)(point=p,iterations=100)